# SQD postprocessing for ATP

In [18]:
fragment = "atp_0_be2_f4"

all_adapt_iterations = [1, 2, 3]

## Setup

In [19]:
import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})
import numpy as np
import pickle
import glob

import pyscf.tools
from pyscf import ao2mo

import collections
from functools import partial
import os
import pickle

from qiskit.primitives import BitArray
from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

In [20]:
def transform_bitstring(bits):
    """
    Convert a given bitstring from Openfermion convention 
    (alternating alpha/beta, big endian) to Qiskit (all alpha
    then all beta, little endian).
    """

    left = [bits[i] for i in range(len(bits)) if i % 2 == 1]   # beta
    right = [bits[i] for i in range(len(bits)) if i % 2 == 0]  # alpha

    # Reverse each half
    left.reverse()
    right.reverse()

    # Concatenate
    return ''.join(left + right)

In [21]:
# import qiskit.visualization


# qiskit.visualization.plot_histogram(
#     all_counts_hardware[0],
#     # target_string=hartree_fock_bitstring,
#     # sort="hamming",
#     number_to_keep=10,
#     figsize=(7, 8),
#     title=computer.name,
# )

In [22]:
circuit_dir = "circuits"
hamiltonian_dir = "hamiltonians"

In [23]:
fcidump = pyscf.tools.fcidump.read(f"{hamiltonian_dir}/{fragment}.fcidump")

Parsing hamiltonians/atp_0_be2_f4.fcidump


In [24]:
n_orbitals = fcidump.get("NORB")
num_electrons = fcidump.get("NELEC")
ecore = fcidump.get("ECORE")
h1 = fcidump.get("H1")
h2 = fcidump.get("H2")
h2 = ao2mo.restore(1, h2, n_orbitals)

In [25]:
nqubits = 2 * n_orbitals

In [26]:
results_dir = "results"

In [27]:
all_counts = []
fnames = []

for adapt_iterations in all_adapt_iterations:
    fname = glob.glob(f"{results_dir}/{fragment}/*{adapt_iterations:03d}*.qasm*")[0]
    counts = pickle.load(
        open(f"{fname}", "rb")
    )
    mode_order = pickle.load(
        open(f"{circuit_dir}/{fragment}/{fragment}_mode_order_{adapt_iterations:03d}_adaptiterations.pkl", "rb")
    )
    qubit_order = pickle.load(
        open(f"{circuit_dir}/{fragment}/{fragment}_qubit_order_{adapt_iterations:03d}_adaptiterations.pkl", "rb")
    )

    measurement_outcomes = counts
    permuted_outcomes = {}
    for original_bitstring in measurement_outcomes.keys():
        qubit_permuted_bitstring = "".join([original_bitstring[qubit_order.index(n)] for n in range(nqubits)])
        mode_permuted_bitstring = "".join([qubit_permuted_bitstring[mode_order.index(n)] for n in range(nqubits)])

        final_permuted_bitstring = transform_bitstring(mode_permuted_bitstring)
        permuted_outcomes[final_permuted_bitstring[::]] = measurement_outcomes[original_bitstring]
    
    counts = permuted_outcomes
    all_counts.append(counts)

    print("ADAPT iteration", adapt_iterations)
    max_key = max(counts, key=counts.get)
    print(f'Most common bitstring: {max_key} with count {counts[max_key]}')
    print(f'Total number of bitstrings: {len(counts)}')
    print(f"Total number of samples:", sum(counts.values()))

ADAPT iteration 1
Most common bitstring: 0000000000000000111111111111111100000000000000001111111111111111 with count 41125
Total number of bitstrings: 1926
Total number of samples: 100000
ADAPT iteration 2
Most common bitstring: 0000000000000000111111111111111100000000000000001111111111111111 with count 37918
Total number of bitstrings: 2613
Total number of samples: 100000
ADAPT iteration 3
Most common bitstring: 0000000000000000111111111111111100000000000000001111111101111111 with count 23041
Total number of bitstrings: 7272
Total number of samples: 100000


In [28]:
# TODO: Implement strategies to cap the number of shots when concatenating.
counts = collections.Counter()
for c in all_counts:
    for bitstring, count in c.items():
        counts[bitstring] += count

In [29]:
sum(counts.values())

300000

In [30]:
bit_array = BitArray.from_counts(counts)

In [ ]:
energy_tol = 1e-8
occupancies_tol = 1e-8
carryover_threshold = 1e-5

sci_solver = partial(solve_sci_batch, spin_sq=0, max_cycle=10000)
result_history = []

def callback(results: list[SCIResult]):
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + ecore}")
        print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")


result = diagonalize_fermionic_hamiltonian(
    one_body_tensor=h1,
    two_body_tensor=h2,
    bit_array=bit_array,
    samples_per_batch=500,
    norb=n_orbitals,
    nelec=(num_electrons // 2, num_electrons // 2),
    num_batches=2,
    energy_tol=energy_tol,
    occupancies_tol=occupancies_tol,
    max_iterations=100,
    sci_solver=sci_solver,
    symmetrize_spin=True,
    carryover_threshold=carryover_threshold,
    callback=callback,
)

Iteration 1
	Subsample 0
		Energy: -261.81985665136665
		Subspace dimension: 51529
	Subsample 1
		Energy: -261.81466852490337
		Subspace dimension: 55696


In [15]:
min_e = [
    min(result, key=lambda res: res.energy).energy + ecore
    for result in result_history
]
min_e

[np.float64(-261.7724553638967),
 np.float64(-261.79274132561227),
 np.float64(-261.80045246597285),
 np.float64(-261.80993016195146),
 np.float64(-261.81179061215096),
 np.float64(-261.818916686501),
 np.float64(-261.82084844780707),
 np.float64(-261.82085631325),
 np.float64(-261.8211667973232),
 np.float64(-261.82273122520394),
 np.float64(-261.8232078886701),
 np.float64(-261.82385588153636),
 np.float64(-261.82529091791406),
 np.float64(-261.82742054154295),
 np.float64(-261.8282846784449),
 np.float64(-261.82829575263884),
 np.float64(-261.82967073134705),
 np.float64(-261.83046759485114),
 np.float64(-261.8316545028493),
 np.float64(-261.8316905364642),
 np.float64(-261.83169765814796),
 np.float64(-261.831955743108),
 np.float64(-261.8343317095521),
 np.float64(-261.8344014504106),
 np.float64(-261.8347597955042),
 np.float64(-261.83508202516146),
 np.float64(-261.8351093326595),
 np.float64(-261.8351125533764),
 np.float64(-261.8351449095565),
 np.float64(-261.8352832521711),


In [16]:
iterations_key = "_".join(map(str, all_adapt_iterations))
save_name = f"sqd_energies_{iterations_key}_adaptiterations.txt"
save_name

'sqd_energies_3_adaptiterations.txt'

In [17]:
np.savetxt(save_name, min_e)